# This program find the decay rate after n days for given activity and also calculates the time required to get number of events for the new rate

# DECAY Rate Equation
# $ R(t)=R_{0}e^{-\lambda t} $
# $ t_{1/2}=\frac{ln(2)}{\lambda} $
# $ R(t)=R_{0}e^{-\frac{ln(2)\cdot t}{t_{1/2}}} $
- $ R_{0} $ (initial rate)
- $ R(t)$ (rate after t seconds)
- $ T_{1/2} $ (half life)



# Realistic rate approach
# $$ R(A) = \gamma A (1+\alpha A t)e^{-\alpha A t }$$


# STEPS
- get optimal rate $R_{0}$ from realistic rate equation (refer: th228rate.ipynb)
- get the decay rate after particular time (t) with decay rate equation considering the half life 
- get R(A) for subsequent rate decay rate R(t) where A is the decay rate using the same $\gamma$, $\alpha$, & $t$ used for  optimal equation values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

# Results from optimal rate

In [ ]:
alpha  = 0.06819796206
beta   = 0.0044755
gamma  = 1.768248e-05
t = 0.01 #drift time (10 ms)

In [ ]:
th228_half_life= 1.9116*365            #1.9116 year into days
th228_initial_activity=2137.32         #optimal activity from simulation of 5M events


print(f"""
Th228 half life       : {th228_half_life} d
Th228 optimal activity: {th228_initial_activity} Bq
""")



In [ ]:
def get_rate_after(initial_activity,half_life,t):
    '''
    returns the rate after time t  for given initial activity and half life: half_life
    Note: t and half_need to have the same unit
    
    '''
    exp_part=np.exp(- np.log(2)*t/half_life)
    return initial_activity*exp_part

#function to find the realistic rate
def get_realistic_rate(A,gamma,alpha,t):
    '''
    returns the realistic rate  for given A, alpha, gamma, and t
    '''
    com=alpha*A*t
    exp_part=np.exp(-com)
    # print(f"""
    # common  : {com}
    # exp_part: {exp_part}
    # """)
    #rate=gamma*A*(1+alpha*A*t)*math.exp(-alpha*A*t)
    rate=gamma*A*(1+com)*exp_part
    #print(f"rate : {rate}")
    return rate
    

In [ ]:
# def get_all_plot(df_1):
#     """
#     returns three plots 
#     """
#     ax1=df_1['decay_rate(Bq)'].plot()
#     ax1.set_xlabel('days')
#     ax1.set_ylabel('decay_rate [Bq]')
#     plt.plot()
#     plt.show()

#     ax2=df_1['optimal_rate(Hz)'].plot()
#     ax2.set_xlabel('days')
#     ax2.set_ylabel('Optimal_rate [Hz]')
#     plt.plot()
#     plt.show()
    

#     ax3=df_1['100_events_time(hr)'].plot()
#     ax3.set_xlabel('days')
#     ax3.set_ylabel('100_events_time [Hr]')
#     plt.plot()
#     plt.show()


In [ ]:
def get_all_plot(df_1,da,yr,save_fig=False):
    """
    returns three plots 
    """
    
    fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True,
                                    figsize=(18, 6))
    days=df_1['days(d)']
    
    ax0.plot(days,df_1['decay_rate(Bq)'])
    ax0.set_xlabel("days")
    ax0.set_ylabel('decay_rate [Bq]')
    
    ax1.plot(days,df_1['optimal_rate(Hz)'])
    ax1.set_xlabel('days')
    ax1.set_ylabel('Optimal_rate [Hz]')
    ax2.plot(days,df_1['100_events_time(hr)'])
    ax2.set_xlabel('days')
    ax2.set_ylabel('100_events_time [Hr]')
    
    plt.suptitle(f"""
    increment days: {da}
    total time    :{yr} yrs
    """)
    plt.plot()
    save_name=f'yr-{yr}-per-{da}.pdf'
    if save_fig:
        print(f"saving the figure with name: {save_name}")
        plt.savefig(save_name)
    plt.show()


In [ ]:
#single function

def get_all_single(increment_days=1,yrs=5):
    """
    returns a dataframe with columns 'days(d)', 'decay_rate(Bq)', 'optimal_rate(Hz)', 'optimal_time(s)' and,
       '100_events_time(hr)' for increment_days and years.
    """
    days=[];rate=[];time=[]

    # 5yrs increment by 1 day
    print(f"""
    ===================================================
    Th228 half life       : {th228_half_life:10} d
    Th228 optimal activity: {th228_initial_activity:10} Bq
    ===================================================
    """)

    print(f"""
    Total time: {yrs} yrs
    increment : {increment_days} d

    """)
    for i in range(1,365*yrs,increment_days):
        rate_realistic=get_rate_after(th228_initial_activity,th228_half_life,i)
        days.append(i);rate.append(rate_realistic);
        debug=False
        if (i-1)%200==0 and debug==True:
            print(f"""
            day       : {i} day(s)
            decay rate: {rate_realistic} Hz
            time      : {t_realistic} hr
            """)
    #optimal rate
    optimal_rate=np.array([get_realistic_rate(i,gamma,alpha,t) for i in rate])
    
    #dataframe
    df_1=pd.DataFrame()
    df_1['days(d)']=days
    df_1['decay_rate(Bq)']=rate
    df_1['optimal_rate(Hz)']=optimal_rate
    df_1['optimal_time(s)']=1/df_1['optimal_rate(Hz)']
    df_1['100_events_time(hr)']=100*df_1['optimal_time(s)']/(60*60)
    
    print(f"                                    DATA FRAME:\n{df_1.head().to_string()}")
    print(f".........................................")
    print(f"{df_1.tail().to_string()}")

    
    get_all_plot(df_1,increment_days,yrs,save_fig=True)
    
    #return df_1

In [ ]:
#[1]+list(range(15,5*365,15))

In [ ]:
#single function to find total deployment time

def get_total_deployment_time(increment_days=1,th228_initial_activity=2137.32,yrs=5):
    """
    returns a dataframe with columns 'optimal_activity(Bq)','increment_days(d)', 'total_deployment_time(yr)','total_100_events_time(hr)'
       
    """
    days=[];rate=[];time=[]

    # 5yrs increment by increment_days 
    print(f"""
    ===================================================
    Th228 half life       : {th228_half_life:10} d
    Th228 optimal activity: {th228_initial_activity:10} Bq
    ===================================================
    """)
    
    print(f"""
    Total time: {yrs} yrs
    increment : {increment_days} d

    """)

   
    for i in range(1,365*yrs,increment_days):
        rate_realistic=get_rate_after(th228_initial_activity,th228_half_life,i)
        days.append(i);rate.append(rate_realistic);
        # debug=False
        # if (i)%200==0 and debug==True:
        #     print(f"""
        #     day       : {i} day(s)
        #     decay rate: {rate_realistic} Hz
        #     time      : {t_realistic} hr
        #     """)
    #optimal rate
    optimal_rate=np.array([get_realistic_rate(i,gamma,alpha,t) for i in rate])
    
    #dataframe
    df_1=pd.DataFrame()
    df_1['days(d)']=days
    df_1['decay_rate(Bq)']=rate
    df_1['optimal_rate(Hz)']=optimal_rate
    df_1['optimal_time(s)']=1/df_1['optimal_rate(Hz)']
    df_1['100_events_time(hr)']=100*df_1['optimal_time(s)']/(60*60)
    
    print(f"                                    DATA FRAME:\n{df_1.head().to_string()}")
    print(f".........................................")
    print(f"{df_1.tail().to_string()}")
    
    total_100_events_time=df_1['100_events_time(hr)'].sum().round(2)
    print(f"""
    total_100_events_time:{total_100_events_time} hr

    """)
   

    
    #get_all_plot(df_1,increment_days,yrs,save_fig=True)
    return_list=[th228_initial_activity,increment_days,yrs,total_100_events_time]
    print(f"Return List: {return_list}")
    return return_list
    
    #return df_1

In [ ]:
yrs=5 #total deployment yrs
#days_list=[1]+list(range(15,365*yrs,15))
days_list=[1,2,5,15,30]
activity_list=list(range(500,10500,500))
final_list=[]
for act in activity_list:
    for i in days_list:
        final_list.append(get_total_deployment_time(increment_days=i,th228_initial_activity=act,yrs=yrs))

In [ ]:
df_f=pd.DataFrame(final_list,columns=['initial_activity(Bq)','increment_days(d)','total_deployment_time(yr)','total_100_events_time(hr)'])
df_f

In [ ]:
min_val=df_f['total_100_events_time(hr)'].min()
min_val

In [ ]:
min_idx=df_f['total_100_events_time(hr)'].argmin()
min_idx

In [ ]:
min_activity=df_f.loc[min_idx]
min_activity

In [ ]:
#group by
grouped=df_f.groupby('initial_activity(Bq)')

In [ ]:
group_list=list(grouped.groups)
group_list

In [ ]:
for i in group_list:
    print(f"with activity: {i}")
    g=grouped.get_group(i).plot(x='increment_days(d)',y='total_100_events_time(hr)',kind='scatter',figsize=(30,10),grid=True,legend=True,logy=True,title=f'initial_activity:{i} Bq')
    plt.show()

In [ ]:
for i in group_list:
    print(grouped.get_group(i).to_string())

In [ ]:
for i in [1,5,15,30,60,120]:
    get_all_single(increment_days=i)

# STOP HERE

In [ ]:
ax1=df_1['decay_rate(Bq)'].plot()
ax1.set_xlabel('days')
ax1.set_ylabel('decay_rate [Bq]')
plt.plot()
plt.show()

ax2=df_1['optimal_rate(Hz)'].plot()
ax2.set_xlabel('days')
ax2.set_ylabel('Optimal_rate [Hz]')
plt.plot()
plt.show()


ax3=df_1['100_events_time(hr)'].plot()
ax3.set_xlabel('days')
ax3.set_ylabel('100_events_time [Hr]')
plt.plot()
plt.show()


In [ ]:
get_realistic_rate(200.0,gamma,alpha,t)

In [ ]:
#lists

yrs=5         #5 years
increment_days=1  #increment days



days=[];rate=[];time=[]

# 5yrs increment by 1 day
print(f"""
===================================================
Th228 half life       : {th228_half_life} d
Th228 optimal activity: {th228_initial_activity} Bq
===================================================
""")

print(f"""
Total time: {yrs} yrs
increment : {increment_days} d

""")
for i in range(1,365*yrs,increment_days):
    #print(f"working for {i} days")
    rate_realistic=get_rate_after(th228_initial_activity,th228_half_life,i)
    N_events=1000
    to_hrs=60*60
    t_realistic=N_events/(rate_realistic*to_hrs)
    days.append(i);rate.append(rate_realistic);time.append(t_realistic)
    if (i-increment_days)/increment_days==0:
        print(f"""
        day       : {i} day(s)
        decay rate: {rate_realistic} Hz
        time      : {t_realistic} hr
        """)

In [ ]:
print(df_1.columns)

In [ ]:
df_1.columns

In [ ]:
get_all_single()

In [ ]:
#optimal rate
optimal_rate=np.array([get_realistic_rate(i,gamma,alpha,t) for i in rate])
#optimal_rate=[get_realistic_rate(i,gamma,alpha,time) for i in rate]

In [ ]:
optimal_rate

In [ ]:
df_1=pd.DataFrame()
df_1['days(d)']=days
df_1['decay_rate(Bq)']=rate
df_1['optimal_rate(Hz)']=optimal_rate

In [ ]:
df_1

In [ ]:
#df_1['optimal_rate(Hz)']=df_1['decay_rate(Bq)']*gamma*(1+alpha*df_1['decay_rate(Bq)']*A*time)*np.exp(-alpha*df_1['decay_rate(Bq)']*time)
df_1['optimal_time(s)']=1/df_1['optimal_rate(Hz)']

In [ ]:
df_1['100_events_time(hr)']=100*df_1['optimal_time(s)']/(60*60)

In [ ]:
df_1

In [ ]:
df_1.head()

In [ ]:
df_1.tail()

In [ ]:
df_1.columns

In [ ]:
%matplotlib inline
ax1=df_1['decay_rate(Bq)'].plot()
ax1.set_xlabel('days')
ax1.set_ylabel('decay_rate [Bq]')
plt.show()



In [ ]:
ax2=df_1['optimal_rate(Hz)'].plot()
ax2.set_xlabel('days')
ax2.set_ylabel('Optimal_rate [Hz]')
plt.show()

In [ ]:
#%matplotlib qt
ax3=df_1['100_events_time(hr)'].plot()
ax3.set_xlabel('days')
ax3.set_ylabel('100_events_time [Hr]')
plt.show()